## **Goal of Notebook**
In this notebook we will check the gender based on the first name of the founders.  
To do so, we use the api from **gender-api.com**

## **Imports**

In [ ]:
import sys
import json
import csv
import numpy as np
import pandas as pd
import configparser
from pathlib import Path
from urllib.request import urlopen
from pyairtable import Table
sys.path.append(str(Path.cwd().parent))

## **Configs**

In [ ]:
# Read in Airtable Data
config = configparser.ConfigParser()
config.read("./../.config/example.ini")
api_key = config.get('Gender_API', 'GENDER_API_KEY')


## **Initialization**

In [ ]:

uploads_dict = {}
data_tuples = []

founders_col_list = ['founder_record_id', 'founder_first_name']

# import csv with the founders_col_list columns
a_founders_df = pd.read_csv("./output/founders.csv")
founders_df = a_founders_df.copy()

missing_columns = [column for column in founders_col_list if column not in a_founders_df.columns]
a_founders_df[missing_columns] = len(missing_columns) * [np.nan]



## **Main Logic**

In [ ]:
for id, name in zip(a_founders_df['founder_record_id'], a_founders_df['founder_first_name']) :

    url = f"https://gender-api.com/get?name={name}&key={api_key}"
    response = urlopen(url)
    decoded = response.read().decode('utf-8')
    data = json.loads(decoded)
    # print( "Gender: " + data["gender"])
    data_tuples.append((id, data["gender"], name))


upload_dicts_list = [
    {'id': record_id, 'fields': {'founder_first_name':name, 'ga_gender': gender}}
    for record_id, gender, name in data_tuples
]


In [ ]:
# Define the output CSV file path
csv_file_path = "./output/founders_data.csv"

# Flatten the upload_dicts_list for CSV export
flattened_data = [
    {'id': entry['id'], **entry['fields']} for entry in upload_dicts_list
]

# Extract fieldnames dynamically
fieldnames = flattened_data[0].keys() if flattened_data else []

# Write data to the CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()  # Write the header row
    writer.writerows(flattened_data)  # Write the data rows
